In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from skimage.feature import canny
from sklearn.preprocessing import LabelEncoder

In [2]:
# EDGE BASED SEGMENTATION
def preprocess1(image):
  # resizing
  image = cv2.resize(image,(96,96))
  # gray scaling
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  return image

In [3]:
rootdir = 'massey_dataset'
images = []
labels = []

for img in os.listdir(rootdir):
    label=img.split('_')[1]
    img = os.path.join(rootdir, img)
    image = cv2.imread(img)
    image = preprocess1(image)
    images.append(image)
    labels.append(label)
    
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)


In [4]:
X = np.array(images)
y = np.array(labels) 
from sklearn.model_selection import train_test_split
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 200
kmeans = learn_visual_words(train_images, num_words)


In [6]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.54      0.56      0.55        25
         1.0       0.59      0.45      0.51        22
         2.0       0.64      0.80      0.71        20
         3.0       0.77      0.87      0.82        23
         4.0       0.68      0.81      0.74        21
         5.0       0.95      0.90      0.93        21
         6.0       0.64      0.67      0.65        21
         7.0       0.78      0.78      0.78        18
         8.0       0.67      0.74      0.70        19
         9.0       0.68      0.88      0.77        17
        10.0       0.67      0.86      0.75        14
        11.0       0.95      0.77      0.85        26
        12.0       0.86      0.71      0.77        17
        13.0       0.57      0.60      0.59        20
        14.0       0.58      0.71      0.64        21
        15.0       0.78      0.88      0.82        16
        16.0       0.76      0.73      0.74        22
        17.0       0.89    

In [7]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 500
kmeans = learn_visual_words(train_images, num_words)


In [8]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.74      0.80      0.77        25
         1.0       0.78      0.82      0.80        22
         2.0       0.71      0.85      0.77        20
         3.0       0.92      0.96      0.94        23
         4.0       0.74      0.95      0.83        21
         5.0       0.95      0.95      0.95        21
         6.0       0.67      0.57      0.62        21
         7.0       0.94      0.83      0.88        18
         8.0       0.89      0.89      0.89        19
         9.0       1.00      0.88      0.94        17
        10.0       0.88      1.00      0.93        14
        11.0       1.00      0.96      0.98        26
        12.0       0.93      0.82      0.87        17
        13.0       0.71      0.75      0.73        20
        14.0       0.90      0.90      0.90        21
        15.0       0.94      1.00      0.97        16
        16.0       1.00      0.86      0.93        22
        17.0       0.91    

In [9]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 1000
kmeans = learn_visual_words(train_images, num_words)


In [10]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.71      0.80      0.75        25
         1.0       0.84      0.73      0.78        22
         2.0       0.85      0.85      0.85        20
         3.0       0.96      1.00      0.98        23
         4.0       0.86      0.90      0.88        21
         5.0       0.84      1.00      0.91        21
         6.0       0.82      0.67      0.74        21
         7.0       0.90      1.00      0.95        18
         8.0       0.85      0.89      0.87        19
         9.0       1.00      0.94      0.97        17
        10.0       0.93      1.00      0.97        14
        11.0       1.00      0.96      0.98        26
        12.0       0.94      1.00      0.97        17
        13.0       0.85      0.85      0.85        20
        14.0       0.95      0.95      0.95        21
        15.0       0.94      1.00      0.97        16
        16.0       1.00      1.00      1.00        22
        17.0       1.00    

In [5]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 700
kmeans = learn_visual_words(train_images, num_words)


In [6]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.82      0.92      0.87        25
         1.0       0.81      0.77      0.79        22
         2.0       0.77      0.85      0.81        20
         3.0       1.00      1.00      1.00        23
         4.0       0.77      0.95      0.85        21
         5.0       0.95      0.90      0.93        21
         6.0       0.87      0.62      0.72        21
         7.0       0.88      0.83      0.86        18
         8.0       0.86      0.95      0.90        19
         9.0       1.00      0.94      0.97        17
        10.0       0.93      1.00      0.97        14
        11.0       1.00      0.88      0.94        26
        12.0       0.83      0.88      0.86        17
        13.0       0.89      0.80      0.84        20
        14.0       0.86      0.90      0.88        21
        15.0       0.94      0.94      0.94        16
        16.0       1.00      1.00      1.00        22
        17.0       0.96    

In [7]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 1200
kmeans = learn_visual_words(train_images, num_words)


In [8]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.84      0.84      0.84        25
         1.0       0.87      0.91      0.89        22
         2.0       0.81      0.85      0.83        20
         3.0       1.00      1.00      1.00        23
         4.0       0.91      0.95      0.93        21
         5.0       0.91      0.95      0.93        21
         6.0       0.81      0.62      0.70        21
         7.0       0.85      0.94      0.89        18
         8.0       1.00      0.95      0.97        19
         9.0       0.94      1.00      0.97        17
        10.0       0.93      1.00      0.97        14
        11.0       1.00      0.92      0.96        26
        12.0       0.94      0.94      0.94        17
        13.0       0.90      0.90      0.90        20
        14.0       0.90      0.90      0.90        21
        15.0       1.00      1.00      1.00        16
        16.0       0.96      1.00      0.98        22
        17.0       1.00    

In [9]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 1600
kmeans = learn_visual_words(train_images, num_words)


In [10]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.87      0.80      0.83        25
         1.0       0.87      0.91      0.89        22
         2.0       0.85      0.85      0.85        20
         3.0       1.00      1.00      1.00        23
         4.0       0.95      1.00      0.98        21
         5.0       0.95      1.00      0.98        21
         6.0       0.79      0.71      0.75        21
         7.0       0.86      1.00      0.92        18
         8.0       0.95      0.95      0.95        19
         9.0       1.00      1.00      1.00        17
        10.0       1.00      1.00      1.00        14
        11.0       1.00      0.92      0.96        26
        12.0       0.94      0.88      0.91        17
        13.0       0.94      0.85      0.89        20
        14.0       1.00      0.95      0.98        21
        15.0       1.00      1.00      1.00        16
        16.0       0.96      1.00      0.98        22
        17.0       1.00    